# Text Generation models

- GPT-4o
- GPT-4o-mini
- GPT-4o with image inpus
- GPT-4o-search-preview

## Chat Completion API

In [1]:
# !pip install -U openai
# !pip install -U tiktoken

In [2]:
import os
import openai
import sys
sys.path.append('./')

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

- 메시지는 메시지 개체의 배열이어야 하며, 각 개체에는role(“system", “user" 또는 “assistant")과 content가 있습니다. 대화는 하나의 메시지만큼 짧을 수도 있고 여러 번 주고받을 수도 있습니다.
- 대화는 먼저 system 메시지로 형식화되고 이어서 user 메시지와 assistanc 메시지가 교대로 표시됩니다.  
- system 메시지는 assistant의 동작을 설정하는 데 도움이 됩니다. 예를 들어, assitant의 성격을 수정하거나 대화 전반에 걸쳐 assistant가 어떻게 행동해야 하는지에 대한 구체적인 지침을 제공할 수 있습니다. 그러나 system 메시지는 선택 사항이며 system 메시지가 없는 모델의 동작은 "당신은 도움이 되는 조수입니다"와 같은 일반적인 메시지를 사용하는 것과 유사할 가능성이 높습니다.  
- 
User 메시지는 assistant가 응답할 요청이나 설명을 제공합니다. Assistant 메시지는 이전 assistant 응답을 저장하지만 원하는 동작의 예를 제공하기 위해 user가 작성할 수도 있습니다  .- 
사용자 지침이 이전 메시지를 참조할 때 대화 기록을 포함하는 것이 중요합니다. 위의 예에서 사용자의 마지막 질문인 " Where was it played?" 는 2020년 월드 시리즈에 대한 이전 메시지의 context 에서만 의미가 있습니다. 모델에는 과거 요청에 대한 기억이 없기 때문에 모든 관련 정보는 각 요청의 대화 기록의 일부로 제공되어야 합니다. 대화가 모델의 토큰 제한에 맞지 않으면 어떤 방식으로든 줄여야 합니다 .



In [3]:
from openai import OpenAI
client = OpenAI()

Model = "gpt-4o-mini"

### temperature가 출력에 미치는 영향
- 낮은 온도(예: 0.1): 값이 낮을수록 모델의 출력은 더욱 결정적이고 집중적입니다. 모델이 가장 높은 확률의 다음 단어를 선택할 가능성이 더 높기 때문에 응답이 더 예측 가능합니다. 이 설정은 사실적 답변이나 데이터 추출과 같이 정확하고 일관된 출력이 필요한 애플리케이션에 유용합니다.

- 고온(예: 0.9): 값이 높을수록 모델의 출력은 더 무작위적이고 창의적이 됩니다. 모델이 덜 가능성 있는 단어를 선택할 가능성이 높기 때문에 응답이 더 다양합니다. 이 설정은 스토리 생성, 브레인스토밍 또는 대화와 같이 변화가 필요한 창의적인 작업에 유용합니다.

In [4]:
response = client.chat.completions.create(
  model=Model,
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "2020년 코리안 시리즈를 우승한 야구팀이 어디?"},
    {"role": "assistant", "content": "2020년 한국시리즈(Korean Series) 우승 야구팀은 NC 다이노스(NC Dinos)입니다."},
    {"role": "user", "content": "마지막 게임이 어디에서 열렸어?"}
  ],
  temperature=0,
)

In [5]:
response.to_dict()

{'id': 'chatcmpl-BZmjNDtSJkoCHJmRTliGPpmu8zrwp',
 'choices': [{'finish_reason': 'stop',
   'index': 0,
   'logprobs': None,
   'message': {'content': '2020년 한국시리즈 마지막 게임은 창원NC파크에서 열렸습니다. NC 다이노스가 홈구장에서 두산 베어스를 상대로 우승을 차지했습니다.',
    'refusal': None,
    'role': 'assistant',
    'annotations': []}}],
 'created': 1747868501,
 'model': 'gpt-4o-mini-2024-07-18',
 'object': 'chat.completion',
 'service_tier': 'default',
 'system_fingerprint': 'fp_0392822090',
 'usage': {'completion_tokens': 42,
  'prompt_tokens': 81,
  'total_tokens': 123,
  'completion_tokens_details': {'accepted_prediction_tokens': 0,
   'audio_tokens': 0,
   'reasoning_tokens': 0,
   'rejected_prediction_tokens': 0},
  'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}}

In [6]:
print(response.choices[0].message.content)

2020년 한국시리즈 마지막 게임은 창원NC파크에서 열렸습니다. NC 다이노스가 홈구장에서 두산 베어스를 상대로 우승을 차지했습니다.


### RAG(Retrieval Augmented Generation)을 이용한 질문 답변

In [7]:
import requests
from bs4 import BeautifulSoup

# Wikipedia 페이지의 URL
#url = "https://en.wikipedia.org/wiki/Curling_at_the_2022_Winter_Olympics"
url = "https://namu.wiki/w/2022%20%EB%B2%A0%EC%9D%B4%EC%A7%95%20%EB%8F%99%EA%B3%84%EC%98%AC%EB%A6%BC%ED%94%BD/%EC%BB%AC%EB%A7%81"

# 페이지 내용 가져오기
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

# 페이지 내용 중에서 필요한 부분 추출 (예시로 본문 전체를 가져옵니다)
wikipedia_article_on_curling = soup.get_text()
wikipedia_article_on_curling

'\n\n\n2022 베이징 동계올림픽/컬링 - 나무위키\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n최근 변경최근 토론특수 기능 \xa02022 베이징 동계올림픽/컬링 최근 수정 시각: 2025-04-15 06:39:394편집토론역사분류2022 베이징 동계올림픽동계올림픽/컬링2022베이징 동계올림픽 [ 펼치기 · 접기 ]주요 정보유치 과정 · 경기 종목 · 선수 목록 · 대회 진행 (개막식 · 폐막식) · 대한민국 대표팀 · 평가  문제점 및 사건 사고 (대규모 외교적 보이콧 · 문화공정 논란ㆍ쇼트트랙 편파판정 논란 (반응) · 카밀라 발리예바 도핑 적발 사건) 개별 종목 쇼트트랙 스피드 스케이팅 ·  피겨 스케이팅 ·  컬링 ·  루지 ·  아이스하키대회 관련 기타 문서마스코트 빙둔둔 · 주경기장 베이징국가체육장 · 패럴림픽 2022 베이징 동계패럴림픽 · 개막 선언자 시진핑 · 성화 점화자 디니걸 이라무장 · 자오자원一起向未来Together For a Shared Future함께하는 미래로2022베이징 동계패럴림픽 [ 펼치기 · 접기 ]주요 정보유치 과정 · 대한민국 대표팀대회 관련 기타 문서마스코트 쉐룽룽 · 주경기장 베이징국가체육장 · 올림픽 2022 베이징 동계올림픽 · 개막 선언자 시진핑 · 성화 점화자 리두안一起向未来Together For a Shared Future함께하는 미래로2022 베이징 동계올림픽 컬링날짜2022년 2월 2일 ~ 2월 20일경기장국립 아쿠아틱 센터종목믹스더블, 남자, 여자챔피언믹스더블 이탈리아남자 스웨덴여자 영국← 20182026 →\xa01. 소개2. 경기 일정3. 출전권3.1. 상세4. 참가 선수 명단4.1. 남자4.1.1. 각 팀 스킵 간략 정보4.2. 여자4.2.1. 각 팀 스킵 간략 정보4.3. 믹스더블4.3.1. 각 팀 간략 정보5. 경기 결과5.1. 믹스더블5.1.1. 라운드 로빈5.1.2. 결선 토너먼트5.2. 남자5.2.1. 라운드 로빈5.2.2. 결선 토너먼트5.3

In [8]:
query = f"""다음 질문에 답하려면 2022년 동계 올림픽에 관한 아래 기사를 사용하세요. 답을 찾을 수 없으면 "모르겠어요."라고 답하세요.

Article:
\"\"\"
{wikipedia_article_on_curling}
\"\"\"

질문: 2022년 동계 올림픽 컬링 금메달을 획득한 선수는 누구인가요?"""

response = client.chat.completions.create(
    messages=[
        {'role': 'system', 'content': '2022년 동계올림픽에 관한 질문에 답변합니다..'},
        {'role': 'user', 'content': query},
    ],
    model=Model,
    temperature=0,
)

print(response.choices[0].message.content)

2022년 동계 올림픽 컬링에서 금메달을 획득한 선수는 다음과 같습니다:

- 믹스더블: 이탈리아
- 남자부: 스웨덴
- 여자부: 영국


### 모든 응답에는 'finish_reason'이 포함되며, 'finish_reason'의 가능한 값은 다음과 같습니다:
- stop: API가 완전한 메시지를 반환하거나, stop 매개변수를 통해 제공된 중단 시퀀스 중 하나에 의해 메시지가 종료됨
- length: max_tokens 매개변수 또는 토큰 제한으로 인해 완전하지 않은 모델 출력
- function_call: 모델이 함수를 호출하기로 결정함

In [9]:
print(response.choices[0].finish_reason)

stop


## Token 관리
- "ChatGPT is great!“ 는 6개의 토큰으로 인코딩됩니다 $\rightarrow$ ["Chat", "G", "PT", " is", " great", "!"]
- 예를 들어, API 호출이 메시지 입력에서 10개의 토큰을 사용하고 메시지 출력에서 ​​20개의 토큰을 받은 경우 30개의 토큰에 대한 요금이 청구됩니다.


In [10]:
print(response.usage)
print(response.usage.prompt_tokens)

CompletionUsage(completion_tokens=52, prompt_tokens=9058, total_tokens=9110, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0))
9058


In [11]:
import tiktoken

encoding = tiktoken.encoding_for_model(Model)

In [12]:
messages=[
    {"role": "system", "content": "You are a helpful assistant designed to output JSON."},
    {"role": "user", "content": "2020년 코리안 시리즈를 우승한 야구팀이 어디?"},
    {"role": "assistant", "content": "2020년 한국시리즈(Korean Series) 우승 야구팀은 NC 다이노스(NC Dinos)입니다."},
    {"role": "user", "content": "마지막 게임이 어디에서 열렸어? 한국어로 답해줘"}
  ]

num_tokens = 0
for message in messages:
  num_tokens += 4  # every message follows <im_start>{role/name}\n{content}<im_end>\n
  for key, value in message.items():
      num_tokens += len(encoding.encode(value))
      print(value)
      print(encoding.encode(value))

num_tokens

system
[17360]
You are a helpful assistant designed to output JSON.
[3575, 553, 261, 10297, 29186, 6884, 316, 4733, 8205, 13]
user
[1428]
2020년 코리안 시리즈를 우승한 야구팀이 어디?
[1323, 15, 12622, 43824, 4960, 14307, 12468, 186042, 4831, 26090, 42643, 3748, 109887, 10997, 91214, 2186, 154074, 30]
assistant
[173781]
2020년 한국시리즈(Korean Series) 우승 야구팀은 NC 다이노스(NC Dinos)입니다.
[1323, 15, 12622, 52971, 5637, 186042, 28797, 99693, 14932, 8, 26090, 42643, 109887, 10997, 91214, 4740, 28895, 14367, 2186, 21464, 5648, 10564, 34, 415, 8200, 8, 27001, 13]
user
[1428]
마지막 게임이 어디에서 열렸어? 한국어로 답해줘
[11630, 118016, 55377, 2186, 154074, 11440, 49496, 76892, 5959, 30, 52971, 5959, 3710, 107393, 5650, 153545]


92

## 재현 가능한 출력 
- SEED 매개 변수 사용

- temperature 설정
  
    - 온도 = 0.0: 모델은 매번 동일한 입력에 대해 동일한 출력을 생성하므로 완전히 결정적입니다. 이는 가장 예측 가능하고 안정적인 응답이 필요할 때 유용합니다.   
    - 0.0과 1.0 사이의 온도: 이것은 가장 일반적으로 사용되는 범위로, 창의성과 결정론 사이의 균형을 허용합니다.   
    - 온도 > 1.0: 가능하지만 출력이 너무 무작위적이어서 대부분의 애플리케이션에 덜 유용할 수 있습니다. 일반적으로 권장되지 않습니다.

In [13]:
from IPython.display import display, HTML

Model = "gpt-4o-mini"
SEED = 123

topic = "화성으로의 여행"
system_message = "당신은 짧은 이야기를 생성하는 도움이 되는 도우미입니다."
user_request = f"{topic}에 관한 짧은 이야기를 한국어로 생성하세요."

try:
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_request},
    ]

    response = openai.chat.completions.create(
        model=Model,
        messages=messages,
        seed=SEED,
        max_tokens=1000,
        temperature=0.0,
    )

    response_content = response.choices[0].message.content
    system_fingerprint = response.system_fingerprint  # 특정 API 응답을 식별하고 추적하는 데 도움 되는 고유 식별자
    completion_tokens = (
        response.usage.total_tokens - response.usage.prompt_tokens
    )
    finish_reason = response.choices[0].finish_reason

    table = f"""
    <table>
    <tr><th style="text-align: left;">Response</th><td style="text-align: left;">{response_content}</td></tr>
    <tr><th style="text-align: left;">System Fingerprint</th><td style="text-align: left;">{system_fingerprint}</td></tr>
    <tr><th style="text-align: left;">Number of completion tokens</th><td style="text-align: left;">{completion_tokens}</td></tr>
    <tr><th style="text-align: left;">finish reason</th><td style="text-align: left;">{finish_reason}</td></tr>
    </table>
    """
    display(HTML(table))
except Exception as e:
    print(f"An error occurred: {e}")

Response,"제목: 붉은 행성의 꿈 2035년, 인류는 드디어 화성으로의 여행을 시작했다. 지구에서 출발한 우주선 '아리랑 1호'는 6개월의 긴 여정을 거쳐 붉은 행성에 도착했다. 승무원들은 지구에서의 일상과는 전혀 다른 풍경에 감탄하며, 화성의 붉은 모래와 거대한 협곡을 바라보았다. 탐사팀의 리더인 수진은 화성의 대기와 환경을 조사하는 임무를 맡았다. 그녀는 우주복을 입고 화성의 표면에 첫 발을 내디뎠다. 그 순간, 그녀의 가슴은 설렘과 두려움으로 가득 찼다. ""이곳은 인류의 새로운 집이 될 수 있을까?""라는 생각이 머리를 스쳤다. 팀원들은 화성의 토양 샘플을 채취하고, 고대의 물 흔적을 찾기 위해 협곡을 탐험했다. 수진은 화성의 신비로운 풍경 속에서 과거의 생명체가 존재했을지도 모른다는 생각에 가슴이 뛰었다. 그들은 화성의 지표면에 작은 식물을 심고, 지구와의 연결을 위해 메시지를 남겼다. ""우리는 여기 있습니다. 우리는 당신을 잊지 않을 것입니다."" 여행의 마지막 날, 수진은 화성의 일몰을 바라보며 마음속으로 다짐했다. ""우리는 이곳에서 새로운 시작을 할 수 있을 거야."" 그녀는 지구로 돌아가면 화성의 이야기를 전하고, 더 많은 사람들이 이 붉은 행성을 꿈꾸게 만들겠다고 결심했다. 아리랑 1호가 지구로 돌아가는 동안, 수진은 화성의 아름다움과 가능성을 잊지 않겠다고 다짐했다. 그리고 그녀의 마음속에는 화성의 붉은 모래가 영원히 남아 있었다."
System Fingerprint,fp_54eb4bd693
Number of completion tokens,447
finish reason,stop


In [14]:
response.to_dict()

{'id': 'chatcmpl-BZmjR5NZGwKYhhJkpJOn4JX8BNWHJ',
 'choices': [{'finish_reason': 'stop',
   'index': 0,
   'logprobs': None,
   'message': {'content': '제목: 붉은 행성의 꿈\n\n2035년, 인류는 드디어 화성으로의 여행을 시작했다. 지구에서 출발한 우주선 \'아리랑 1호\'는 6개월의 긴 여정을 거쳐 붉은 행성에 도착했다. 승무원들은 지구에서의 일상과는 전혀 다른 풍경에 감탄하며, 화성의 붉은 모래와 거대한 협곡을 바라보았다.\n\n탐사팀의 리더인 수진은 화성의 대기와 환경을 조사하는 임무를 맡았다. 그녀는 우주복을 입고 화성의 표면에 첫 발을 내디뎠다. 그 순간, 그녀의 가슴은 설렘과 두려움으로 가득 찼다. "이곳은 인류의 새로운 집이 될 수 있을까?"라는 생각이 머리를 스쳤다.\n\n팀원들은 화성의 토양 샘플을 채취하고, 고대의 물 흔적을 찾기 위해 협곡을 탐험했다. 수진은 화성의 신비로운 풍경 속에서 과거의 생명체가 존재했을지도 모른다는 생각에 가슴이 뛰었다. 그들은 화성의 지표면에 작은 식물을 심고, 지구와의 연결을 위해 메시지를 남겼다. "우리는 여기 있습니다. 우리는 당신을 잊지 않을 것입니다."\n\n여행의 마지막 날, 수진은 화성의 일몰을 바라보며 마음속으로 다짐했다. "우리는 이곳에서 새로운 시작을 할 수 있을 거야." 그녀는 지구로 돌아가면 화성의 이야기를 전하고, 더 많은 사람들이 이 붉은 행성을 꿈꾸게 만들겠다고 결심했다.\n\n아리랑 1호가 지구로 돌아가는 동안, 수진은 화성의 아름다움과 가능성을 잊지 않겠다고 다짐했다. 그리고 그녀의 마음속에는 화성의 붉은 모래가 영원히 남아 있었다.',
    'refusal': None,
    'role': 'assistant',
    'annotations': []}}],
 'created': 1747868505,
 'model': 'gpt-4o-mini-2024-07-18

### 모델에 인터넷의 최신 정보에 대한 액세스 권한을 부여
- `gpt-4o-search-preview` 또는 `gpt-4o-mini-search-preview` 모델 사용

In [15]:
completion = client.chat.completions.create(
    model="gpt-4o-mini-search-preview",
    web_search_options={},
    messages=[
        {
            "role": "user",
            "content": "오늘 한국에서 나온 가장 큰 뉴스가 뭐야?",
        }
    ],
)

print(completion.choices[0].message.content)

2025년 5월 21일 현재, 한국에서 가장 큰 뉴스는 다음과 같습니다:

1. **한국, 첫 군사 정찰위성 발사 성공**: 한국은 최근 첫 군사 정찰위성을 발사하여 우주 감시 능력을 강화했습니다. ([voakorea.com](https://www.voakorea.com/a/7397677.html?utm_source=openai))

2. **미국 국가정보국장, 한국 방문 제외 논란**: 미국의 정보기관 수장인 털시 개버드 국가정보국장이 일본을 방문하면서 한국을 제외한 사실이 알려져 논란이 일고 있습니다. ([news.nate.com](https://news.nate.com/view/20250324n13606?utm_source=openai))

3. **사우디아라비아, 한국 드라마 '닭강정'에 대한 비판**: 넷플릭스 오리지널 시리즈 '닭강정'이 사우디아라비아 왕실을 언급한 장면으로 인해 아랍권 시청자들로부터 비판을 받고 있습니다. ([maxmovieen.com](https://www.maxmovieen.com/max-star/article/48705/?utm_source=openai))

4. **한국, 3D 프린팅 건설기술 특허출원 증가율 세계 1위**: 최근 5년간 한국의 3D 프린팅 건설기술 특허출원 증가율이 세계에서 가장 높은 것으로 나타났습니다. ([m.news.nate.com](https://m.news.nate.com/view/20241027n06573?utm_source=openai))

5. **한국 경제, 물가 안정세 확대 및 내수 회복 조짐**: 최근 한국 경제는 물가 안정세가 확대되고 내수 회복의 조짐을 보이고 있습니다. ([news.nate.com](https://news.nate.com/view/20241018n22797?utm_source=openai))

이러한 소식들은 현재 한국에서 주목받고 있는 주요 뉴스입니다. 


## 실습 : 위의 messages 내용을 각자 수정해 가면서 API 실행